In [1]:
import os
from pathlib import Path

def get_project_path():
    if "__file__" in globals():
        # Running as a .py script
        this_file_type = "python script"
        project_path = Path(__file__).resolve().parent.parent.parent.parent.parent
    else:
        # Presume running in a Jupyter notebook; use CWD’s great-grandparent
        this_file_type = "notebook"
        project_path = Path.cwd().resolve().parent
    return project_path, this_file_type

project_path, this_file_type = get_project_path()
print("project_path:", project_path)
print("file tyep:", this_file_type)

project_path: D:\projects\_create_new_project
file tyep: notebook


The following is to create a json for the file structures of the project

In [ ]:
import json
import os

project_structure = {
    "<project_name>": {
        "backend": {
            "apps": {
                "javascript": {"modules": {}},
                "python": {"modules": {}},
                "r": {"modules": {}}
            }
        },
        "batch_files": {},
        "powershell":{},
        "logs":{},
        "data": {
            "in": {},
            "out": {}
        },
        "frontend": {
            "testdata": {
                "in": {},
                "out": {}
            },
            "apps": {
                "javascript": {
                    "modules": {}
                }
            },
            "config": {},
            "index.html": None
        },
        "notebooks": {},
        ".gitignore": None,
        ".env": None,
        ".flaskenv": None,
        "README.md": None
    }
}

def write_structure_to_json(json_path):
    os.makedirs(os.path.dirname(json_path), exist_ok=True)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(project_structure, f, indent=2)




In [ ]:
# Example usage:
json_path = f"{project_path}/data/in/_project_structure.json"  # Change this to your desired path
write_structure_to_json(json_path)

The following is a function to read such structure.josn, replace <project_name> with the input project name, and create such project under the parent folder

In [4]:
import os
import json
import shutil

def create_project_from_structure(json_path, project_name, project_parent_path, ignore_str=None):
    # Read JSON structure
    with open(json_path, 'r', encoding='utf-8') as f:
        structure = json.load(f)

    if not ignore_str:
        ignore_str="node_modules/\ndata/\ntestdata/\nvenv/\nbackend/app/sas/MyMacros.sas\n.env\n*.yml\ngoogle_sheets_api_service_accoun_key.json\noauth 2 client credential.json\nmdbtools-win/\nbchomes-analysis-8791ad06c20a.json\ncredentials.json\nconfig_ngrok.js\nngrok/\nvenv*/\nlibs_external/\n.flaskenv\ncompleted/\nqueue/\nprocessing/\nerror/\nremote_control/"

    # Replace placeholder project name
    if "<project_name>" not in structure:
        raise ValueError("JSON must have '<project_name>' as the top-level key.")
    structure = {project_name: structure["<project_name>"]}

    # Create full project path under the parent
    root = os.path.join(project_parent_path, project_name)
    os.makedirs(root, exist_ok=True)

    created_files = []
    skipped_files = []

    def write_file_if_missing(file_path, content):
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        if os.path.exists(file_path):
            skipped_files.append(file_path)
            return
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
        created_files.append(file_path)

    def copy_file_if_missing(src_file, dst_file):
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        if os.path.exists(dst_file):
            skipped_files.append(dst_file)
            return
        shutil.copy2(src_file, dst_file)
        created_files.append(dst_file)

    def copy_dir_no_overwrite(src_dir, dst_dir):
        if not os.path.isdir(src_dir):
            return
        for current_dir, _, files in os.walk(src_dir):
            rel = os.path.relpath(current_dir, src_dir)
            target_dir = dst_dir if rel == '.' else os.path.join(dst_dir, rel)
            os.makedirs(target_dir, exist_ok=True)
            for filename in files:
                src_file = os.path.join(current_dir, filename)
                dst_file = os.path.join(target_dir, filename)
                copy_file_if_missing(src_file, dst_file)

    def create_items(base_path, item_dict, ignore_str):
        for name, content in item_dict.items():
            item_path = os.path.join(base_path, name)
            if content is None:
                file_content = ''
                if name == '.gitignore':
                    file_content = ignore_str
                elif name == 'README.md':
                    file_content = f'# {project_name}\n\npython -m pip install --upgrade pip\npip install nbformat ipynbname\npip install --upgrade ipywidgets jupyter\n'
                elif name == '.flaskenv':
                    file_content = 'FLASK_APP =\nFLASK_RUN_PORT =\n'
                elif name == 'index.html':
                    title_text = project_name.replace(' ', '_').replace('-', '_')
                    file_content = f"""<!DOCTYPE html>
<html lang=\"en\">
<head>
    <meta charset=\"UTF-8\">
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\">
    <title>{title_text}</title>
</head>
<body>
</body>
</html>
"""
                write_file_if_missing(item_path, file_content)
            elif isinstance(content, dict):
                os.makedirs(item_path, exist_ok=True)
                create_items(item_path, content, ignore_str)

    create_items(root, structure[project_name], ignore_str)

    # Copy the full batch_files folder into the created project root without overwriting files
    source_batch_dir = os.path.abspath(
        os.path.join(os.path.dirname(json_path), '..', '..', 'batch_files')
    )
    target_batch_dir = os.path.join(root, 'batch_files')
    copy_dir_no_overwrite(source_batch_dir, target_batch_dir)

    # Copy utility notebooks into the created project's notebooks folder without overwriting files
    source_notebooks_dir = os.path.abspath(
        os.path.join(os.path.dirname(json_path), '..', '..', 'notebooks', '_general')
    )
    target_notebooks_dir = os.path.join(root, 'notebooks', '_general')
    notebook_files = [
        '_00_make_files_map.ipynb',
        '_01_download_hf_models.ipynb',
        '_02_install_packages_cuda_windows.ipynb',
        '_03_export_notebook_to_py.ipynb',
    ]
    for notebook_file in notebook_files:
        src_notebook = os.path.join(source_notebooks_dir, notebook_file)
        if not os.path.isfile(src_notebook):
            print(f'Missing source notebook, skipped: {src_notebook}')
            continue
        dst_notebook = os.path.join(target_notebooks_dir, notebook_file)
        copy_file_if_missing(src_notebook, dst_notebook)

    print(f'Project created/updated at: {root}')
    if skipped_files:
        print('Skipped existing files (not overwritten):')
        for skipped in skipped_files:
            print(f'- {skipped}')
    else:
        print('No files were skipped.')


In [5]:
project_structure_json_path = f"{project_path}/data/in/_project_structure.json"  # Change this to your desired path
project_name = "transcript" 
project_parent_path = r"D:\projects"
create_project_from_structure(project_structure_json_path, project_name, project_parent_path)
# create_project_from_structure("C:/path/to/structure.json", "my_project", "C:/projects/my_project")


Project created/updated at: D:\projects\transcript
No files were skipped.
